## Text Classification - Emotion Detection 

This sample shows how to fine tune a model to detect emotions using emotion dataset and deploy it to an endpoint for real time inference.

### Training data
We will use the [emotion](https://huggingface.co/datasets/dair-ai/emotion) dataset a copy of which is available in the `azureml` system registry for easy access.

### Model
Models that can perform the `fill-mask` task are generally good candidates to fine tune for `text-classification`. We will list all models of the `fill-mask` type, from which you can pick one. If you opened this notebook from a specific model card, copy past the model `Asset ID`. Optionally, if you need to fine tune a model that is available on HuggingFace, but not available in `azureml` system registry, you can either [import](https://github.com/Azure/azureml-examples) the model or use the `huggingface_id` parameter to use a model directly from HuggingFace. [Learn more]().

### Outline
* Setup pre-requisites such as compute.
* Pick a model to fine tune.
* Pick and explore training data.
* Configure the fine tuning job.
* Run the fine tuning job.
* Register the fine tuned model. 
* Deploy the fine tuned model for real time inference.
* Clean up resources. 



### 1. Setup pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry
* Set an optional experiment name
* Check or create compute. A single GPU node can have multiple GPU cards. For example, in one node of `Standard_ND40rs_v2` there are 8 NVIDIA V100 GPUs while in `Standard_NC12s_v3`, there are 2 NVIDIA V100 GPUs. Refer to the [docs](https://learn.microsoft.com/en-us/azure/virtual-machines/sizes-gpu) for this information. The number of GPU cards per node is set in the param `gpus_per_node` below. Setting this value correctly will ensure utilization of all GPUs in the node. The recommended GPU compute SKUs can be found [here](https://learn.microsoft.com/en-us/azure/virtual-machines/ncv3-series) and [here](https://learn.microsoft.com/en-us/azure/virtual-machines/ndv2-series).

In [8]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, ClientSecretCredential
from azure.ai.ml.entities import AmlCompute

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

workspace_ml_client = MLClient(
        credential,
        subscription_id =  "21d8f407-c4c4-452e-87a4-e609bfb86248", #"<SUBSCRIPTION_ID>"
        resource_group_name =  "rg-contoso-819prod", #"<RESOURCE_GROUP>",
        workspace_name =  "mlw-contoso-819prod", #"WORKSPACE_NAME>",
)
 
registry_ml_client = MLClient(
    credential,
    registry_name="azureml-preview",
)

experiment_name = "text-classification-emotion-detection"

compute_cluster = "gpu-cluster-big"
try:
    workspace_ml_client.compute.get(compute_cluster)
except Exception as ex:
    compute = AmlCompute(
        name = compute_cluster, # If you already have a gpu cluster, mention it here.
        size= "Standard_ND40rs_v2",
        max_instances= 2 # For multi node training set this to an integer value more than 1
    )
    workspace_ml_client.compute.begin_create_or_update(compute).wait()

gpus_per_node = 2 # This is the number of GPUs in a single node of the selected 'vm_size' compute


### 2. Pick a model to fine tune

We will query the `azureml` system registry and list all models of the type `fill-mask`. Any of these models will work for `text-classification`, but in this example, we use `bert-base-uncased`. If you have opened this notebook for a specific mode, replace the model name and version accordingly. 

In [9]:
models = registry_ml_client.models.list()
for model in models:
    versions=registry_ml_client.models.list(model.name) # replace this with get the latest version?
    for version in versions:
        if (version.tags['task'] == 'fill-mask'):
            print ("Model name: {0}, version: {1}".format(version.name, version.version))
        break       

Model name: distilbert-base-uncased, version: 3
Model name: distilroberta-base, version: 3
Model name: roberta-large, version: 3
Model name: camembert-base, version: 3
Model name: microsoft-deberta-base, version: 3
Model name: bert-large-uncased, version: 3
Model name: microsoft-deberta-xlarge, version: 3
Model name: distilbert-base-cased, version: 3
Model name: bert-large-cased, version: 3
Model name: bert-base-cased, version: 3
Model name: microsoft-deberta-large, version: 3
Model name: bert-base-uncased, version: 3
Model name: roberta-base, version: 3


In [10]:
model_name = "bert-base-uncased"
model_version = "3"
print ("\n\nUsing model name: {0}, version: {1} for fine tuning".format(model_name, model_version))



Using model name: bert-base-uncased, version: 3 for fine tuning


### 3. Prepare the dataset for to fine tune
> This notebook pulls from HuggingFace datasets but we will change this to point to system registry after we onboard sample data to system registry and fine tune component supports data splitting

Start by fetching dataset label names. The actual data contains label numeric categories so we will use this metadata to add a column that contains actual label names when we download the dataset.

In [11]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
import pandas as pd
# toto - this data asset should be loaded from the system registry
data_asset = workspace_ml_client.data.get(name="emotion", version=1)
print(data_asset)

# todo - show some sample data from the data asset
# df = pd.read_json(data_asset.path, lines=True)

creation_context:
  created_at: '2023-03-15T23:54:19.132158+00:00'
  created_by: Manoj Bableshwar
  created_by_type: User
  last_modified_at: '2023-03-15T23:54:19.147819+00:00'
id: /subscriptions/21d8f407-c4c4-452e-87a4-e609bfb86248/resourceGroups/rg-contoso-819prod/providers/Microsoft.MachineLearningServices/workspaces/mlw-contoso-819prod/data/emotion/versions/1
name: emotion
path: azureml://subscriptions/21d8f407-c4c4-452e-87a4-e609bfb86248/resourcegroups/rg-contoso-819prod/workspaces/mlw-contoso-819prod/datastores/workspaceblobstore/paths/LocalUpload/d4561647b85625a246688ae8f566c7fa/emotion-unsplit.json
properties: {}
tags: {}
type: uri_file
version: '1'



### 4. Submit the fine tuning job using the the model and data as inputs
 
Create the job that uses the `text-classification` pipeline component. [Learn more]() about all the parameters supported for fine tuning.

In [19]:
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities import CommandComponent, PipelineComponent, Job, Component
from azure.ai.ml import PyTorchDistribution, Input

# fetch the pipeline component
pipeline_component_func = registry_ml_client.components.get(name="textclassificationsinglelabel_pipelinecomponent", version="0.0.13")

# temporary registry until split_dataset is available in fine tune component
registry_data_ml_client = MLClient(
    credential,
    registry_name="sample-data",
)
split_dataset_func = registry_data_ml_client.components.get(name="split_dataset", version="0.0.11")

# define the pipeline job
@pipeline()
def create_pipeline():
    split_data_job = split_dataset_func(
        data_file = Input(type="uri_file", path=data_asset.path),
        train_split = 0.05, # dataset has 50k+ rows, so picking a small number for sample pipeline
        validation_split = 0.005, # 10% of train split
        test_split = 0.005, # 10% of train split
    )
    split_data_job.compute = compute_cluster

    finetuning_job = pipeline_component_func( 
        huggingface_id = model_name, # this needs to change to use model from system registry
        compute_model_selector = compute_cluster,
        compute_preprocess = compute_cluster,
        compute_finetune = compute_cluster,
        compute_model_evaluation = compute_cluster,
        train_file_path = split_data_job.outputs.train_file, 
        valid_file_path = split_data_job.outputs.validation_file,
        test_file_path = split_data_job.outputs.test_file,
        sentence1_key = "text", # picked up by visualizing the sample data in step 3
        label_key = "label_string", # picked up by visualizing the sample data in step 3
        test_data_input_column_names = "text", # picked up by visualizing the sample data in step 3
        process_count_per_instance_finetune = gpus_per_node, # set to the number of GPUs available in the compute
        epochs = 2,
        learning_rate = 2e-5, 
    )
    return {
        "trained_model": finetuning_job.outputs.mlflow_model_folder_finetune
    }

pipeline_object = create_pipeline()
pipeline_object.display_name =  "text-classification-using-" + model_name

Submit the job

In [20]:
pipeline_job = workspace_ml_client.jobs.create_or_update(pipeline_object, experiment_name=experiment_name)
workspace_ml_client.jobs.stream(pipeline_job.name)

RunId: gray_basket_ldlxr7bjlx
Web View: https://ml.azure.com/runs/gray_basket_ldlxr7bjlx?wsid=/subscriptions/21d8f407-c4c4-452e-87a4-e609bfb86248/resourcegroups/rg-contoso-819prod/workspaces/mlw-contoso-819prod

Streaming logs/azureml/executionlogs.txt

[2023-03-16 05:28:50Z] Submitting 2 runs, first five are: 4a45a41a:d29a1872-cc6d-40f5-9794-518301334583,5a7a9693:fd4b32b7-045c-4101-b25e-fc927f8a63a7


### 5. Register the fine tuned model with the workspace

We will register the model from the output of the fine tuning job. This will track lineage between the fine tuned model and the fine tuning job. The fine tuning job, further, tracks lineage to the foundation model, data and training code.

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
# check if the `trained_model` output is available
print ("pipeline job outputs: ", workspace_ml_client.jobs.get(pipeline_job.name).outputs)

# fetch the model from pipeline job output - not working, hence fetching from fine tune child job
# model_path_from_job = ("azureml://jobs/{0}/outputs/{1}".format(pipeline_job.name, "trained_model"))

for level1_job in workspace_ml_client.jobs.list(parent_job_name=pipeline_job.name): # pipeline component job
    for level2_job in workspace_ml_client.jobs.list(parent_job_name=level1_job.name): # pipeline component subgraph job (not shown in UI)
        for level3_job in workspace_ml_client.jobs.list(parent_job_name=level2_job.name): # child jobs
            if (level3_job.display_name == "finetune"):
                model_path_from_job = ("azureml://jobs/{0}/outputs/{1}".format(level3_job.name, "mlflow_model_folder"))

finetuned_model_name = model_name + "-emotion-detection"
print("path to register model: ", model_path_from_job)
#prepare_to_register_model = Model(
#    path=model_path_from_job,
#    type=AssetTypes.MLFLOW_MODEL,
#    name=finetuned_model_name
#    version=1,
#    description=model_name + " fine tuned model for emotion detection"
#)
#print(prepare_to_register_model)
# register the model from pipeline job output 
# registered_model = workspace_ml_client.models.create_or_update(prepare_to_register_model)


pipeline job outputs:  {'trained_model': <azure.ai.ml.entities._job.pipeline._io.base.PipelineOutput object at 0x7f62988e60a0>}
path to register model:  azureml://jobs/4765c75a-871a-476f-ac5b-5fba7cd01263/outputs/mlflow_model_folder


In [ ]:
# Use cli to register model, with path from the above output until 
! az ml model create --path azureml://jobs/4765c75a-871a-476f-ac5b-5fba7cd01263/outputs/mlflow_model_folder --name bert-base-uncased-emotion-detection --version 1 --type mlflow_model


Class FeatureStoreOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FeaturesetOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FeaturestoreEntityOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
{
  "creation_context": {
    "created_at": "2023-03-15T05:48:59.688470+00:00",
    "created_by": "Manoj Bableshwar",
    "created_by_type": "User",
    "last_modified_at": "2023-03-15T05:48:59.688470+00:00",
    "last_modified_by": "Manoj Bableshwar",
    "last_modified_by_type": "User"
  },
  "flavors": {
    "hftransformers": {
      "code": "",
      "hf_pretrained_class": "BertForSequenceClassification",
      "huggingface_id": "bert-base-uncased",
      "model_data": "data",
      "pytorch_version": "1.11.0",
    

### 6. Deploy the fine tuned model to an online endpoint
Online endpoints give a durable REST API that can be used to integrate with applications that need to use the model.

In [ ]:
import time, sys
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment

finetuned_model_name = "bert-base-uncased" + "-emotion-detection"

registered_model = workspace_ml_client.models.get(name=finetuned_model_name, version=1)

timestamp = str(int(time.time())) # endpoint names need to be unique in a region, hence using timestamp to create unique endpoint name
# Create online endpoint
online_endpoint_name = "emotion-" + timestamp
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for " + registered_model.name + ", fine tuned model for emotion detection",
    auth_mode="key"
)
workspace_ml_client.begin_create_or_update(endpoint).wait()

In [ ]:
# create a deployment
demo_deployment = ManagedOnlineDeployment(
    name="demo",
    endpoint_name=online_endpoint_name,
    model=registered_model.id,
    instance_type="Standard_DS2_v2",
    instance_count=1,
)
workspace_ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
endpoint.traffic = {"demo": 100}
workspace_ml_client.begin_create_or_update(endpoint).result()

Check: endpoint emotion-1678894711 exists
data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


HttpResponseError: (BadRequest) The request is invalid.
Code: BadRequest
Message: The request is invalid.
Exception Details:	(InferencingClientCreateDeploymentFailed) InferencingClient HttpRequest error, error detail: {"errors":{"VmSize":["Not enough quota available for Standard_DS2_v2 in SubscriptionId ed2cab61-14cc-4fb3-ac23-d72609214cfd. Current usage/limit: 98/100. Additional needed: 4 Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-outofquota"]},"type":"https://tools.ietf.org/html/rfc7231#section-6.5.1","title":"One or more validation errors occurred.","status":400,"traceId":"00-ab61d316fbbc35369a98123cfda92535-0d1503e90d490258-01"}
	Code: InferencingClientCreateDeploymentFailed
	Message: InferencingClient HttpRequest error, error detail: {"errors":{"VmSize":["Not enough quota available for Standard_DS2_v2 in SubscriptionId ed2cab61-14cc-4fb3-ac23-d72609214cfd. Current usage/limit: 98/100. Additional needed: 4 Please see troubleshooting guide, availab

### 7. Test the endpoint with sample data

We will fetch some sample data from the test data and submit to online endpoint for inference.

In [ ]:
# todo